# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAIN_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 5000

# Read and Load features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAIN_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_test_data[-5:]

array([[5793, 2350,    0,    2],
       [5793, 2356,    0,    4],
       [5793, 2382,    0,    1],
       [5793, 2405,    0,    3],
       [5793, 2494,    0,    1]])

In [7]:
print('Number of rows in training file:', sparse_train_data.shape[0])
print('Number of rows in test file:', sparse_test_data.shape[0])

Number of rows in training file: 258372
Number of rows in test file: 117781


In [8]:
print('Number of emails in training file:', np.unique(sparse_train_data[:, 0]).size)

Number of emails in training file: 4013


In [9]:
print('Number of emails in testing file:', np.unique(sparse_test_data[:, 0]).size)

Number of emails in testing file: 1724


## How to create an empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix full matrix from a sparse matrix. Return a pandas DataFrame.
    Keywords arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in sparse matrix. Default: 1st column
    word_idx -- position of the word id in sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default 4th column
    """

    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, nr_words))
    doc_id_names = np.unique(sparse_matrix[:, doc_idx])

    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)

    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence

    full_matrix.set_index('DOC_ID', inplace=True)

    return full_matrix

In [16]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 13.8 s


In [17]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
full_train_data.shape

(4013, 2501)

# Training the Naive Bayes Model
## Calculating the Probability of Spam

In [20]:
full_train_data.CATEGORY.size

4013

In [21]:
full_train_data.CATEGORY.sum()

1248

In [22]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of Spam is:', prob_spam)

Probability of Spam is: 0.310989284824321


## Total Number of Words / Tokens

In [23]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
email_lengths = full_train_features.sum(axis=1)
email_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5789     18
5790     72
5791     46
5794     57
5795    264
Length: 4013, dtype: int64

In [25]:
# Total Word Count
total_wc = email_lengths.sum()
total_wc

429241

## Number of Tokens in Spam and Ham emails

In [26]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
1885     45
1887     26
1889    115
1890     26
1895    162
Length: 1248, dtype: int64

In [27]:
spam_wc = spam_lengths.sum()
spam_wc

176337

In [28]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths

DOC_ID
1896     82
1898    112
1899     66
1900     49
1901     34
       ... 
5789     18
5790     72
5791     46
5794     57
5795    264
Length: 2765, dtype: int64

In [29]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252904

In [30]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [31]:
total_wc - spam_wc - nonspam_wc

0

In [32]:
print('Average number of words in spam emails: {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Average number of words in ham emails: {:.0f}'.format(nonspam_wc/ham_lengths.shape[0]))

Average number of words in spam emails: 141
Average number of words in ham emails: 91


In [33]:
full_train_features.shape

(4013, 2500)

## Summing the tokens occurring in spam

In [34]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# We do not want zero in our calculations. So we add 1 to each! It is called Laplace Smoothing Technique
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495       9
2496       7
2497       6
2498      26
2499       1
Length: 2500, dtype: int64

## Summing the tokens occurring in ham

In [36]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# We do not want zero in our calculations. So we add 1 to each! It is called Laplace Smoothing Technique
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens

0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495      26
2496      26
2497      25
2498       3
2499      29
Length: 2500, dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the email is spam

In [38]:
# Vocab Size added to deal with effect of Laplace Smoothing Technique which we used before to avoid zeroes!
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam

0       0.012184
1       0.005228
2       0.006805
3       0.011306
4       0.006816
          ...   
2495    0.000050
2496    0.000039
2497    0.000034
2498    0.000145
2499    0.000006
Length: 2500, dtype: float64

In [39]:
prob_tokens_spam.sum()

1.0

## P(Token | Spam) - Probability that a Token Occurs given the email is spam

In [40]:
# Vocab Size added to deal with effect of Laplace Smoothing Technique which we used before to avoid zeroes!
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_spam

0       0.012184
1       0.005228
2       0.006805
3       0.011306
4       0.006816
          ...   
2495    0.000050
2496    0.000039
2497    0.000034
2498    0.000145
2499    0.000006
Length: 2500, dtype: float64

In [41]:
prob_tokens_spam.sum()

1.0

# P(Token) - Probability that Token Occurs

In [42]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all

0       0.017848
1       0.008208
2       0.007595
3       0.006891
4       0.006591
          ...   
2495    0.000077
2496    0.000072
2497    0.000068
2498    0.000063
2499    0.000065
Length: 2500, dtype: float64

In [43]:
prob_tokens_all.sum()

0.9999999999999999

# Save Trained Model

In [44]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [45]:
sparse_test_data.shape

(117781, 4)

In [46]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 7.38 s


In [47]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [48]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)